In [11]:
import pandas as pd
import numpy as np
import os
from utils import loadTabular, loadSequential, dumpCleanedSequential, dumpTabular
X_train, labels = loadTabular()
X_test = loadTabular(isTest=True)
train_seq = loadSequential()
test_seq = loadTabular()

In [16]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
dumpTabular(X_train)
dumpTabular(X_test, True)